This file is where we read in the two different datasets, one from Countlove, and one from the Crowd Counting Consortium.

In [5]:
import numpy as np
import pandas as pd
import re
import datetime
#import all the necessary items
racial = pd.read_csv("racialdata.csv", encoding= 'unicode_escape')
racial.head()
#first read in the countlove data which has been labeled as racialdata.csv

,Date,Location,Attendees,Event (legacy; see tags),Tags,Curated,Source,Total Articles
0,1/15/17,"Bowie State University, Bowie, MD",1500.0,Healthcare,Healthcare; For Affordable Care Act,Yes,http://www.capitalgazette.com/news/ph-ac-cn-ac...,1
1,1/16/17,"Johnson City, TN",300.0,Civil Rights,Civil Rights; For racial justice; Martin Luthe...,Yes,http://www.johnsoncitypress.com/Local/2017/01/...,4
2,1/16/17,"Indianapolis, IN",20.0,Environment,Environment; For wilderness preservation,Yes,http://wishtv.com/2017/01/16/nature-groups-pro...,1
3,1/16/17,"Cincinnati, OH",NaN,Other (Martin Luther King Jr.),Civil Rights; For racial justice; Martin Luthe...,Yes,http://www.cincinnati.com/picture-gallery/news...,1
4,1/18/17,"Hartford, CT",300.0,Healthcare (Pro-Planned Parenthood),Healthcare; For Planned Parenthood,Yes,http://www.realhartford.org/2017/01/19/reprodu...,1


Before we look at any of our other data we need to examine the data we have from count love.

In [9]:
racial.dtypes

Date                         object
Location                     object
Attendees                   float64
Event (legacy; see tags)     object
Tags                         object
Curated                      object
Source                       object
Total Articles                int64
dtype: object

From this observation we see that the only columns we need to change are the date, which we do below.

In [12]:
racial['Date'] = pd.to_datetime(racial['Date'])

In [7]:
#Now we read in each of the files for the months before
#We had to use a specific encoding because of the shape of the files
june_2019 = pd.read_csv("Crowd Estimates June 2019-edit.csv", encoding= 'unicode_escape')
july_2019 = pd.read_csv("Crowd Estimates July 2019-edit.csv", encoding= 'unicode_escape')
august_2019 = pd.read_csv("Crowd Estimates August 2019-edit.csv", encoding= 'unicode_escape')
september_2019 = pd.read_csv("Crowd Estimates September 2019-edit.csv", encoding= 'unicode_escape')
october_2019 = pd.read_csv("Crowd Estimates October 2019-edit.csv", encoding= 'unicode_escape')
november_2019 = pd.read_csv("Crowd Estimates November 2019-edit.csv", encoding= 'unicode_escape')
december_2019 = pd.read_csv("Crowd Estimates December 2019-edit.csv", encoding= 'unicode_escape')
january_2020 = pd.read_csv("Crowd Estimates January 2020-edit.csv", encoding= 'unicode_escape')
march_2020 = pd.read_csv("Crowd Estimates March 2020-edit.csv", encoding= 'unicode_escape')
april_2020 = pd.read_csv("Crowd Estimates April 2020-edit.csv", encoding= 'unicode_escape')


#September had a number of especially difficult items to work with. They encoded different versions for missing
#variables as well as some of the numbers were not configured correctly
september_2019['Pro(2)/Anti(1)'] = september_2019['Pro(2)/Anti(1)'].map({'2':'2','1':'1','0':'0','2.':'2','1.':'1','0.':'0'})
september_2019['ReportedArrests'] = september_2019['ReportedArrests'].replace({'nan':np.nan, 'DEADLINK':np.nan,
                                                                              'DEAD LINK':np.nan,'DEADLINK, doesn\'t allow access to article':np.nan})
september_2019['ReportedParticipantInjuries'] = september_2019['ReportedParticipantInjuries'].replace({'nan':np.nan, 'DEADLINK':np.nan,
                                                                              'DEAD LINK':np.nan,'DEADLINK, doesn\'t allow access to article':np.nan})
september_2019['ReportedPoliceInjuries'] = september_2019['ReportedPoliceInjuries'].replace({'nan':np.nan, 'DEADLINK':np.nan,
                                                                              'DEAD LINK':np.nan,'DEADLINK, doesn\'t allow access to article':np.nan})
september_2019['ReportedPropertyDamage'] = september_2019['ReportedPropertyDamage'].replace({'nan':np.nan, 'DEADLINK':np.nan,
                                                                              'DEAD LINK':np.nan,'DEADLINK, doesn\'t allow access to article':np.nan,
                                                                                            'DEADLINK\t':np.nan})
#imported only the ones under tally and not the individual ones specifically for certain marches - like women's rights
#and light for life

In [8]:
december_2019.head()

,ï»¿CityTown,Location,County,StateTerritory,Country,Date,EstimateText,EstimateLow,BestGuess,EstimateHigh,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,Kansas City,outside the Truman,Jackson,MO,US,2019-12-01,about a dozen,12.0,NaN,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,San Diego,Hillcrest,San Diego,CA,US,2019-12-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,San Francisco,outside Moscone Center,San Francisco,CA,US,2019-12-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Terre Haute,Hellman Hall at St. Benedict Catholic Church,Vigo,IN,US,2019-12-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Washington,outside White House,DC,DC,US,2019-12-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In order to combine both the countlove data and the crowd consortium data together we first need to clean the crowd consortium data and get it in the correct form. We do this by creating a function, shown below.

In [3]:
def add_city(x):
    #take the first column, which is the city name, for some of the datasets there were different figures
    #infront of the word City so we chose to just do it this way so each column could be processed no matter
    #their first column name
    first = x.columns[0]
    
    #next rename the first column
    x.rename(columns = {first: "City"}, inplace = True)
    
    #then drop the columns that the city is null, as these are not values that we can train on
    drop_city = x[x['City'].isnull() == True].index
    x.drop(drop_city, inplace = True)
    
    #next select the columns that we need and want
    x = x.loc[:,['City','Location','County','StateTerritory','Country','Date','EstimateText','EstimateLow','BestGuess','EstimateHigh','AdjustedLow',
  'AdjustedHigh','Actor','Claim','Pro(2)/Anti(1)','EventType','ReportedArrests','ReportedParticipantInjuries',
  'ReportedPoliceInjuries','ReportedPropertyDamage','TownsCities','Events']]
    
    #create a new column called location that will store both the city and the state
    x['Location'] = x['City']
    
    #combine the two values of state and city together into a new column
    for y in list(x.index):
        city = x.loc[y, 'City']
        state = x.loc[y, 'StateTerritory']
        x.loc[y, 'Location'] = str(city) + ', ' + str(state)
    
    #create a new column called event
    x['Event'] = x['Claim']
    
    #the claim column for the crowd consortium data has what the countlove data clasifies as an event together
    #with the claims, so we tried to break those two up, and if the event didn't exit just keep them the same
    for y in list(x.index):
        if ';' in str(x.loc[y,'Claim']):
            split = str(x.loc[y,'Claim']).split(';')
            if len(split) == 2:
                event = split[0]
                claim = split[1]
                x.loc[y,'Claim'] = claim
                x.loc[y,'Event'] = event
            else:
                both = split[0]
                x.loc[y,'Claim'] = both
                x.loc[y,'Event'] = both
    
    #then we dropped the column that we no longer needed because we had created new versions of them that 
    #corresponded with the countlove versions
    x = x.drop(columns = ['City', 'StateTerritory', 'Events', 'TownsCities', 'Country', 
                      'EstimateText', 'BestGuess'])
    
    #finally we edited the values to make them have their proper type for analysis
    x['Date'] = pd.to_datetime(x["Date"])
    x['Pro(2)/Anti(1)'] = pd.to_numeric(x['Pro(2)/Anti(1)'])
    x['ReportedArrests'] = pd.to_numeric(x['ReportedArrests']) 
    x['ReportedParticipantInjuries'] = pd.to_numeric(x['ReportedParticipantInjuries'])
    x['ReportedPoliceInjuries'] = pd.to_numeric(x['ReportedPoliceInjuries'])
    x['ReportedPropertyDamage'] = pd.to_numeric(x['ReportedPropertyDamage'])
    return x

In [4]:
#apply the add_city function to each of the datasets and save them into a new dataset
july_2019new = add_city(july_2019)
august_2019new  = add_city(august_2019)
september_2019new  = add_city(september_2019)
october_2019new = add_city(october_2019)
november_2019new = add_city(november_2019) 
december_2019new = add_city(december_2019) 
january_2020new = add_city(january_2020) 
march_2020new = add_city(march_2020) 
april_2020new = add_city(april_2020)
june_2019new = add_city(june_2019)

Now we can combine the two datasets together

In [7]:
#make a list of the datasets to add to the original set
months_new = [june_2019new, july_2019new, august_2019new,september_2019new, october_2019new, november_2019new, 
          december_2019new, january_2020new, march_2020new ,april_2020new]

#use a for loop to merge each of the dataframes together on their specific columns
for x in months_new:
    racial = racial.merge(x, on = ['Location', 'Date','County', 'EstimateLow','EstimateHigh', 'AdjustedLow','AdjustedHigh','Actor',
 'Claim','Pro(2)/Anti(1)','EventType','ReportedArrests','ReportedParticipantInjuries','ReportedPoliceInjuries',
 'ReportedPropertyDamage','Event'], how = 'left')
    #all other ones merge properly


Now we can clean this dataset and only get the rows that we need

In [8]:
#We won't be able to train using an event that doesn't have any attendees, as this is a value that we intend
#to predict. So we find the columsn where the attendance value is unknown and drop those columns
n = racial[(racial['Attendees'].isnull() == True) & (racial['EstimateHigh'].isnull() == True)].index
racial.drop(n, inplace = True)

#next we check for rows where the column Attendees doesn't have any values but EstimateHigh does have values
#and we add in the values from EstimateHigh to the Attendees column
index_attendee = racial[(racial['Attendees'].isnull() == True) & (racial['EstimateHigh'].isnull() == False)].index
for x in range(len(index_attendee)):
    racial.loc[index_attendee[x], 'Attendees'] = racial.loc[index_attendee[x], 'EstimateHigh']

In [9]:
#next we drop the columns that are no longer needed
racial.drop(columns=['Source','Curated', 'Total Articles','EstimateLow','EstimateHigh', 'AdjustedLow', 'AdjustedHigh'],inplace = True)

In [10]:
racial.head()

,Date,Location,Attendees,Event (legacy; see tags),Tags,County,Actor,Claim,Pro(2)/Anti(1),EventType,ReportedArrests,ReportedParticipantInjuries,ReportedPoliceInjuries,ReportedPropertyDamage,Event
0,2017-01-15,"Bowie State University, Bowie, MD",1500.0,Healthcare,Healthcare; For Affordable Care Act,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-16,"Johnson City, TN",300.0,Civil Rights,Civil Rights; For racial justice; Martin Luthe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-16,"Indianapolis, IN",20.0,Environment,Environment; For wilderness preservation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-18,"Hartford, CT",300.0,Healthcare (Pro-Planned Parenthood),Healthcare; For Planned Parenthood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2017-01-20,"Westlake Park, Seattle, WA",100.0,Executive,Executive; Against president,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now need to get state and city columns, we had these originally from the consortium data but dropped them when we were organizing the data for merging 

In [11]:
#make a function that takes in the location and returns the state
def state(place):
    #split the location on commas
    new = str(place).split(',')
    #check the length of the list, if it is three then we need to take the last value at index 2
    if len(new) == 3:
        state = re.sub(r"^\s+", "", new[2])
        return state
    #if the length is 2 then we take the value at index 1
    elif len(new) == 2:
        state = re.sub(r"^\s+", "", new[1])
        return state
    #and if there is only one value then we just take the single one
    else:
        return new[0]
racial['State'] = racial.apply(lambda row: state(row['Location']), axis=1)

In [12]:
#make a function that takes in the location and returns the city
def city(place):
    #split on commas
    new = str(place).split(',')
    #check length of the list, if it is three then we need to take the middle value at index 1
    if len(new) == 3:
        city = re.sub(r"^\s+", "", new[1])
        return city
    #if it is two then we need to take the first value
    elif len(new) == 2:
        return new[0]
    #if there is only one value, then return np.nan
    else:
        return np.nan
racial['City'] = racial.apply(lambda row: city(row['Location']), axis=1)

There are a couple of cases where there are special locations, which we create a column for below.

In [13]:
#make a function that takes a location and returns the special location
def specific(place):
    #split on commas
    new = str(place).split(',')
    #if the lenght of the place list is 3, then there is a special location so return that location
    if len(new) == 3:
        return new[0]
    #else return np.nan
    else:
        return np.nan
racial['Specific Location'] = racial.apply(lambda row: specific(row['Location']), axis=1)

Now that we have correct states, cities, and special location lists we can filter on the locations we are interested in.

In [14]:
#create a list of cities that we are going to look at
locations = ['New York','Washington','Portland', 'Los Angeles','San Francisco', 'Boston', 'Chicago','Philadelphia', 'Springfield',
'Austin','Salt Lake City','Seattle','Detroit','Phoenix', 'San Diego','Sacramento']   

#create a function that will return True if the city is in the list and False if the city isn't in the list
def include(city):
    if city in locations:
        return 'True'
    else:
        return 'False'
racial['Include'] = racial.apply(lambda row: include(row['City']), axis = 1)      

In [15]:
#make a new dataframe with only the true values
data = racial[racial['Include'] == 'True']

Since our data is clean now we can look at the values we have for each year - the number of rows is listed next to each years dataframe in a comment.

In [16]:
d2017 = datetime.datetime(2017, 12, 31)
data_2017 = data[data['Date'] <= d2017] #782 rows 
d2018 = datetime.datetime(2018, 12, 31)
data_2018 = data[(data['Date'] <= d2018) & (data['Date'] > d2017)] #769 rows 
d2019 = datetime.datetime(2019, 12, 31)
data_2019 = data[(data['Date'] <= d2019) & (data['Date'] > d2018)] #747 rows 
d2020 = datetime.datetime(2020, 12, 31)
data_2020 = data[(data['Date'] <= d2020) & (data['Date'] > d2019)] #903 rows

From these datasets we can observe the different events and tags that were the most popular. From this we will be able to discern which accounts on twitter we should follow and the types of protests that we are more likely to find.

In [18]:
data_2020['Event (legacy; see tags)'].value_counts()

Racial Injustice           536
Other                       94
Healthcare                  74
Civil Rights                43
Executive                   41
International Relations     39
Collective Bargaining       28
Education                   23
Immigration                 12
Environment                  7
Guns                         6
Name: Event (legacy; see tags), dtype: int64

In [19]:
data_2020['Tags'].value_counts()[:20]

Civil Rights; For racial justice; For greater accountability; Police                             449
Healthcare; Against pandemic intervention; Coronavirus                                            62
International; Against war; Iran                                                                  23
Civil Rights; For women's rights; Women's March                                                   12
Education; Against hazardous conditions; Coronavirus                                              12
Civil Rights; For racial justice; For greater accountability; For freedom of assembly; Police     11
Other; For supporting police                                                                      11
Other; Against eviction; Coronavirus                                                               9
Immigration; For compassionate immigration                                                         8
Civil Rights; For racial justice; Martin Luther King, Jr.                                  